In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
import numpy as np

In [2]:
training_data = pd.read_csv('training_twitter_x_y_train.csv')
testing_data = pd.read_csv('test_twitter_x_test.csv')

In [3]:
training_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [4]:
testing_data

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [5]:
training_tweet = list(training_data['text'].values)
training_categories = list(training_data['airline_sentiment'].values)
testing_tweet = list(testing_data['text'].values)

In [6]:
for i,sent in enumerate(training_tweet):
    words = word_tokenize(sent.casefold())
    training_tweet.pop(i)
    training_tweet.insert(i, words)
for i,sent in enumerate(testing_tweet):
    words = word_tokenize(sent.casefold())
    testing_tweet.pop(i)
    testing_tweet.insert(i, words)

In [7]:
documents = []
for i, category in enumerate(training_categories):
    documents.append((training_tweet[i], category))
documents[:5]

[(['@',
   'southwestair',
   'i',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'cancelled',
   'flightled'],
  'negative'),
 (['@',
   'southwestair',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'i',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'flew',
   'ord',
   'to',
   'miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'thanks'],
  'positive'),
 (['@',
   'southwestair',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [8]:
import random
random.shuffle(documents)
documents[:5]

[(['@',
   'americanair',
   'my',
   'flight',
   '1389',
   'from',
   'las',
   'vegas',
   'to',
   'dfw',
   'was',
   'cancelled',
   'flightled',
   '!',
   'i',
   "'ve",
   'been',
   'on',
   'hold',
   'forever',
   'and',
   'i',
   'still',
   'have',
   'not',
   'spoken',
   'to',
   'anyone',
   '!',
   'pls',
   'help'],
  'negative'),
 (['@',
   'united',
   'my',
   'bag',
   'is',
   'lost',
   'and',
   'i',
   'ca',
   "n't",
   'get',
   'ahold',
   'of',
   'anyone',
   'that',
   'can',
   'help'],
  'negative'),
 (['@', 'virginamerica', 'save', 'some', 'for', '871', 'tomorrow', 'am', '!'],
  'neutral'),
 (['@',
   'usairways',
   'forced',
   'sections',
   '4',
   'and',
   '5',
   'to',
   'check',
   'their',
   'carry',
   'on',
   '.',
   'would',
   'have',
   'packed',
   'differently',
   'to',
   'check',
   'my',
   'bag',
   '.',
   'why',
   'even',
   'allow',
   'it',
   '?',
   '#',
   'pissed'],
  'negative'),
 (['@',
   'united',
   'my',
   '

In [9]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import string
stop_words = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stop_words.update(punctuations)
print(stop_words, string.punctuation)

{'themselves', "hasn't", 't', 'haven', 'how', 'ain', 'no', 'after', 'very', 'doesn', 'shan', 'needn', 'they', 'but', 'yourself', 'here', 'other', 'at', '`', 'we', 'won', 'the', "wouldn't", "shan't", 'between', 'their', 'while', "needn't", 'by', 'as', ',', 'for', '#', 'himself', '\\', "she's", '/', 'up', 'a', 'm', "should've", 'itself', 'o', 'have', "'", 'hasn', 'where', 'will', 'these', 'again', '}', "won't", 'your', 'is', 'in', 'further', 'when', ')', 's', 'those', 'mightn', 'who', "you'd", 'its', 'weren', 'of', '.', 'isn', 'having', 'you', 'own', 'shouldn', 'only', '%', 'd', ':', '^', 'once', "you'll", 'down', 'mustn', 'do', 'me', "aren't", 'll', 'now', 'my', 'above', 'our', 'off', 'then', 'her', 'i', 'ours', '>', 're', 'into', "didn't", 'more', 'am', "wasn't", 'below', 'such', 'hadn', 'couldn', "you've", 'him', 'doing', 'all', 'so', 'should', "mightn't", '(', '~', 'aren', 'his', 'just', "you're", '*', 'was', 'theirs', "doesn't", '{', '|', 'being', 'until', "shouldn't", 'there', 'und

In [10]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stop_words:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [12]:
documents = [(clean_review(document), category) for document, category in documents]

In [13]:
print(documents[0])

(['americanair', 'flight', '1389', 'la', 'vega', 'dfw', 'cancel', 'flightled', "'ve", 'hold', 'forever', 'still', 'spoken', 'anyone', 'pls', 'help'], 'negative')


In [14]:
training_documents, testing_documents = train_test_split(documents, random_state=1)

In [15]:
all_words = []
for doc in training_documents:
    all_words += doc[0]

In [16]:
freq = nltk.FreqDist(all_words) # Frequency Distribution i.e the freq of occurance of a particular word
common = freq.most_common(2500) # The most frequent words also referred as most_common(3000)
features = [i[0] for i in common]

In [17]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [18]:
get_feature_dict(training_documents[0][0])

{'flight': False,
 'united': True,
 'americanair': False,
 'usairways': False,
 'southwestair': False,
 'jetblue': False,
 'get': False,
 "n't": False,
 "'s": False,
 'http': False,
 'hour': False,
 'thanks': False,
 'service': False,
 'cancel': False,
 'customer': False,
 'u': False,
 'help': False,
 '...': False,
 'time': False,
 'call': False,
 'bag': False,
 'go': True,
 'wait': False,
 'plane': False,
 'make': True,
 'amp': False,
 'fly': False,
 'hold': False,
 'need': False,
 "'m": False,
 '2': False,
 'thank': False,
 'would': False,
 'still': False,
 'airline': False,
 'try': False,
 'day': False,
 'one': False,
 'take': False,
 'back': False,
 'gate': False,
 'ca': False,
 'virginamerica': False,
 'delayed': False,
 'seat': False,
 'say': False,
 'please': False,
 'bad': True,
 'book': False,
 'flightled': False,
 'delay': False,
 "'ve": True,
 "''": False,
 'like': False,
 '``': False,
 'agent': False,
 'late': False,
 'today': False,
 'change': False,
 'miss': False,
 'phon

In [19]:
training_documents

[(['united',
   'ewr',
   'mco',
   'make',
   'unplanned',
   'land',
   'pressurization',
   'failure',
   'bad',
   'pain',
   "'ve",
   'ever',
   'felt',
   'thought',
   'go',
   'pas'],
  'negative'),
 (['usairways',
   '12',
   'hr',
   'early',
   'send',
   'email',
   'share',
   'experience',
   "n't",
   'want',
   'blasting'],
  'negative'),
 (['americanair',
   'chariot',
   'tonight',
   'angle',
   'look',
   'like',
   '787',
   'http',
   '//t.co/3zzkqwwbjz'],
  'neutral'),
 (['united',
   'good',
   'thing',
   'hold',
   'flight',
   '11',
   'peep',
   'late',
   'flight',
   'connect',
   'flight',
   "'ll",
   'still',
   'make',
   'destination',
   'time'],
  'positive'),
 (['southwestair',
   'know',
   'finally',
   'line',
   'like',
   'cattle',
   'board',
   'sw',
   'flight',
   'original',
   'departure',
   'time',
   'badcustomerservice'],
  'negative'),
 (['americanair',
   'usairways',
   'need',
   'talk',
   'ensure',
   'statement',
   'sent',
 

In [20]:
x_train = [(get_feature_dict(doc), category) for doc, category in training_documents]
x_test = [(get_feature_dict(doc), category) for doc, category in testing_documents]

In [21]:
x_train[0]

({'flight': False,
  'united': True,
  'americanair': False,
  'usairways': False,
  'southwestair': False,
  'jetblue': False,
  'get': False,
  "n't": False,
  "'s": False,
  'http': False,
  'hour': False,
  'thanks': False,
  'service': False,
  'cancel': False,
  'customer': False,
  'u': False,
  'help': False,
  '...': False,
  'time': False,
  'call': False,
  'bag': False,
  'go': True,
  'wait': False,
  'plane': False,
  'make': True,
  'amp': False,
  'fly': False,
  'hold': False,
  'need': False,
  "'m": False,
  '2': False,
  'thank': False,
  'would': False,
  'still': False,
  'airline': False,
  'try': False,
  'day': False,
  'one': False,
  'take': False,
  'back': False,
  'gate': False,
  'ca': False,
  'virginamerica': False,
  'delayed': False,
  'seat': False,
  'say': False,
  'please': False,
  'bad': True,
  'book': False,
  'flightled': False,
  'delay': False,
  "'ve": True,
  "''": False,
  'like': False,
  '``': False,
  'agent': False,
  'late': False,


In [22]:
from nltk import NaiveBayesClassifier

In [23]:
classifier = NaiveBayesClassifier.train(x_train)

In [24]:
nltk.classify.accuracy(classifier, x_test)

0.7712204007285974

In [ ]:
rfc = RandomForestClassifier()
classifier_sklearn1 = SklearnClassifier(rfc)

In [ ]:
classifier_sklearn1.train(x_train)

In [ ]:
nltk.classify.accuracy(classifier_sklearn1, x_test)

In [ ]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [ ]:
svc = SVC(kernel='linear')
classifier_sklearn = SklearnClassifier(svc)

In [ ]:
svc.fit(x_train, y_train)
y_test = svc.predict(x_test)

In [ ]:
classifier_sklearn.train(x_train)

In [ ]:
nltk.classify.accuracy(classifier_sklearn, x_test)